In [ ]:
#11001+12397

In [5]:
import logging
import os
from gensim.models.fasttext import FastText

from gensim.models import KeyedVectors
EMBEDDING_DIR='Model/'
PREPROCESSED_DIR='Data/preprocessed/'
class WordEmbeddingConfig(object):
    """Word2Vec Training parameters"""
    window=10 #Maximum skip length window between words
    emb_dim=200 # Set size of word vectors
    emb_lr=0.05 #learning rate for SGD estimation.
    nepoach=20 #number of training epochs
    nthread=100 #number of training threads
    sample = 0 #Set threshold for occurrence of words. Those that appear with higher frequency in the training data will be randomly down-sampled
    negative = 15 #negative sampling is used with defined negative example
    hs = 0 #0 Use Hierarchical Softmax; default is 0 (not used)
    binary=0 # 0 means not saved as .bin. Change to 1 if allowed to binary format
    sg=0 # 0 means CBOW model is used. Change to 1 to use Skip-gram model
    iterate=10 # Run more training iterations
    minFreq=1 #This will discard words that appear less than minFreq times 
    if sg==0:
        model_name='am_fasttext_cbow_'+str(emb_dim)+'D'
    elif sg==1:
         model_name='am_fasttext_sg_'+str(emb_dim)+'D'

class corpus_sentences(object):# accept sentence stored one per line in list of files inside defined directory
    def __init__(self, dirname):
        self.dirname = dirname
    
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding='utf8'):
                yield line.split()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)


def load_am_word_vectors():
    if WordEmbeddingConfig.sg==0:
            model_type='CBOW'
    else:
        model_type='Skip-gram'        
    if os.path.exists(WordEmbeddingConfig.model_name):
        print('Loading Word2Vec Amharic Pretrained '+model_type+' model with '+str(WordEmbeddingConfig.emb_dim)+' dimension\n') 
        am_model= KeyedVectors.load(WordEmbeddingConfig.model_name)
    else:
        print('Loading Sentences with memory freindly iterator ...\n')
        sentences = corpus_sentences(PREPROCESSED_DIR) # a memory-friendly iterator        
        print('Training FastText '+model_type+' with '+str(WordEmbeddingConfig.emb_dim)+' dimension\n') 
        am_model = FastText(size=WordEmbeddingConfig.emb_dim, window=WordEmbeddingConfig.window, 
                            min_count=WordEmbeddingConfig.minFreq, workers=WordEmbeddingConfig.nthread,sg=WordEmbeddingConfig.sg,
                            iter=WordEmbeddingConfig.iterate,negative=WordEmbeddingConfig.negative,
                            hs=WordEmbeddingConfig.hs)
        am_model.build_vocab(sentences)

        am_model.train(sentences, total_examples=am_model.corpus_count, epochs=am_model.iter)
        #trim unneeded model memory = use (much) less RAM
        am_model.init_sims(replace=True)
        
        #Saving model    
        model_name=EMBEDDING_DIR+WordEmbeddingConfig.model_name
        am_model.save(model_name)        
        
    return am_model    
# am_model=load_am_word_vectors()        

In [6]:
from gensim.models import Word2Vec,KeyedVectors
am_model= KeyedVectors.load('Model/am_fasttext_cbow_200D')
print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ ጅማ".split()))
print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ".split()))
print(am_model.wv.doesnt_match("ጥረቶች ተግባራት ቋንቋዎች ክንዋኔዎች".split()))
print(am_model.wv.most_similar_cosmul(positive=['ችግር', 'አገልግሎቶች'], negative=['አገልግሎት']))
print(am_model.wv.most_similar_cosmul(positive=['ቱርክ', 'አፍሪቃ'], negative=['ኢትዮጵያ']))
print(am_model.wv.most_similar_cosmul(positive=['እናት', 'አባቶች'], negative=['አባት']))
# 			እናቶች					
# 		የክርስቲያኖችን				
# 						ውጤታማነት
print(am_model.wv.most_similar_cosmul(positive=['ክርስቲያን', 'የሙስልሞችን'], negative=['ሙስልም']))
print(am_model.wv.most_similar_cosmul(positive=['ውጤት', 'ምርታማነት'], negative=['ምርት']))
print('Total Vocab: ',len(am_model.wv.vocab))

ጅማ
ብዙ
ቋንቋዎች
[('ችግርች', 0.9998194575309753), ('ችግርዎ', 0.9903538823127747), ('ችግርሽስ', 0.9890170097351074), ('ችግርኮ', 0.9885969758033752), ('ችግርዋ', 0.9845884442329407), ('ችግርር', 0.9830366969108582), ('ችግርፈቺ', 0.9734110236167908), ('ችግርፇ', 0.970687985420227), ('ችግሮች', 0.9634712934494019), ('ችግርግጭት', 0.963415801525116)]
[('ኢስያ', 0.9846081137657166), ('ኒኮስያ', 0.9833387136459351), ('ሮደሽያ', 0.9819197058677673), ('ሪኬትስያ', 0.9689559936523438), ('ጐስያ', 0.9584450125694275), ('ፊሊስያ', 0.9522125124931335), ('ቃስያ', 0.9506744146347046), ('ኋስያ', 0.9458655118942261), ('ሌቲስያ', 0.94571453332901), ('ቱኒስያ', 0.9411526322364807)]
[('ህጣናት', 0.9509854316711426), ('ጣናት', 0.9282034039497375), ('እናእዝናት', 0.9265158176422119), ('ሪናት', 0.918435275554657), ('አንስትሴቶች', 0.9181416630744934), ('ሁናት', 0.9179297089576721), ('ህፀናት', 0.9175921082496643), ('ቅድመአባቶች', 0.9161309599876404), ('ፊናት', 0.9156181812286377), ('ፃናት', 0.915442943572998)]
[('ሳስቶፓስቲያን', 0.8907968997955322), ('ስቲያን', 0.8874554634094238), ('ክግስቲያን', 0.886150717

In [4]:
import logging
import os
from gensim.models import Word2Vec,KeyedVectors
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
EMBEDDING_DIR='Model/'
PREPROCESSED_DIR='Data/preprocessed/'
class WordEmbeddingConfig(object):
    """Word2Vec Training parameters"""
    window=5 #Maximum skip length window between words
    emb_dim=200 # Set size of word vectors
    emb_lr=0.05 #learning rate for SGD estimation.
    nepoach=10 #number of training epochs
    nthread=100 #number of training threads
    sample = 0 #Set threshold for occurrence of words. Those that appear with higher frequency in the training data will be randomly down-sampled
    negative = 10 #negative sampling is used with defined negative example
    hs = 0 #0 Use Hierarchical Softmax; default is 0 (not used)
    binary=0 # 0 means not saved as .bin. Change to 1 if allowed to binary format
    sg=0 # 0 means CBOW model is used. Change to 1 to use Skip-gram model
    iterate=20 # Run more training iterations
    minFreq=5 #This will discard words that appear less than minFreq times 
    if sg==0:
        model_name='5w_10ng_am_w2v_cbow_'+str(emb_dim)+'D'
    elif sg==1:
         model_name='5w_10ng_am_w2v_cbow_'+str(emb_dim)+'D'

class corpus_sentences(object):# accept sentence stored one per line in list of files inside defined directory
    def __init__(self, dirname):
        self.dirname = dirname
    
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding='utf8'):
                yield line.split()
    
def train_w2v_model():
    print('Loading Sentences with memory freindly iterator ...\n')
    sentences = corpus_sentences(PREPROCESSED_DIR) # a memory-friendly iterator 
    if WordEmbeddingConfig.sg==0:
            model_type='CBOW'
    else:
        model_type='Skip-gram'    
    print('Training Word2Vec '+model_type+' with '+str(WordEmbeddingConfig.emb_dim)+' dimension\n') 
    _model = Word2Vec(sentences, size=WordEmbeddingConfig.emb_dim, window=WordEmbeddingConfig.window, 
                            min_count=WordEmbeddingConfig.minFreq, workers=WordEmbeddingConfig.nthread,sg=WordEmbeddingConfig.sg,
                            iter=WordEmbeddingConfig.iterate,negative=WordEmbeddingConfig.negative,sample=WordEmbeddingConfig.sample,
                            hs=WordEmbeddingConfig.hs,sorted_vocab=1)
  
    #trim unneeded model memory = use (much) less RAM
    _model.init_sims(replace=True)
    
    #Saving model   
    model_name=EMBEDDING_DIR+WordEmbeddingConfig.model_name
    _model.save(model_name)

    return _model        
train_w2v_model()

Loading Sentences with memory freindly iterator ...

Training Word2Vec CBOW with 200 dimension



2020-02-16 13:00:46,123 : INFO : collecting all words and their counts
2020-02-16 13:00:46,215 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-02-16 13:00:47,256 : INFO : PROGRESS: at sentence #10000, processed 166643 words, keeping 32480 word types
2020-02-16 13:00:47,826 : INFO : PROGRESS: at sentence #20000, processed 344000 words, keeping 55626 word types
2020-02-16 13:00:48,147 : INFO : PROGRESS: at sentence #30000, processed 463152 words, keeping 73728 word types
2020-02-16 13:00:48,573 : INFO : PROGRESS: at sentence #40000, processed 589187 words, keeping 90713 word types
2020-02-16 13:00:49,325 : INFO : PROGRESS: at sentence #50000, processed 727443 words, keeping 105542 word types
2020-02-16 13:00:50,028 : INFO : PROGRESS: at sentence #60000, processed 893932 words, keeping 120996 word types
2020-02-16 13:00:50,454 : INFO : PROGRESS: at sentence #70000, processed 1021202 words, keeping 131651 word types
2020-02-16 13:00:50,850 : INFO : PROGRESS:

2020-02-16 13:01:15,631 : INFO : PROGRESS: at sentence #710000, processed 9153631 words, keeping 424555 word types
2020-02-16 13:01:16,102 : INFO : PROGRESS: at sentence #720000, processed 9280122 words, keeping 426938 word types
2020-02-16 13:01:16,410 : INFO : PROGRESS: at sentence #730000, processed 9407328 words, keeping 429553 word types
2020-02-16 13:01:16,703 : INFO : PROGRESS: at sentence #740000, processed 9535543 words, keeping 432101 word types
2020-02-16 13:01:17,128 : INFO : PROGRESS: at sentence #750000, processed 9660652 words, keeping 434666 word types
2020-02-16 13:01:17,548 : INFO : PROGRESS: at sentence #760000, processed 9784893 words, keeping 437096 word types
2020-02-16 13:01:17,944 : INFO : PROGRESS: at sentence #770000, processed 9910732 words, keeping 439553 word types
2020-02-16 13:01:18,441 : INFO : PROGRESS: at sentence #780000, processed 10033742 words, keeping 441624 word types
2020-02-16 13:01:18,791 : INFO : PROGRESS: at sentence #790000, processed 10158

2020-02-16 13:01:52,195 : INFO : PROGRESS: at sentence #1420000, processed 22152518 words, keeping 917304 word types
2020-02-16 13:01:52,719 : INFO : PROGRESS: at sentence #1430000, processed 22352518 words, keeping 923272 word types
2020-02-16 13:01:53,296 : INFO : PROGRESS: at sentence #1440000, processed 22552518 words, keeping 928680 word types
2020-02-16 13:01:53,797 : INFO : PROGRESS: at sentence #1450000, processed 22752518 words, keeping 934366 word types
2020-02-16 13:01:54,295 : INFO : PROGRESS: at sentence #1460000, processed 22952518 words, keeping 939573 word types
2020-02-16 13:01:54,849 : INFO : PROGRESS: at sentence #1470000, processed 23152518 words, keeping 945163 word types
2020-02-16 13:01:55,530 : INFO : PROGRESS: at sentence #1480000, processed 23352518 words, keeping 949279 word types
2020-02-16 13:01:56,289 : INFO : PROGRESS: at sentence #1490000, processed 23552518 words, keeping 952874 word types
2020-02-16 13:01:56,710 : INFO : PROGRESS: at sentence #1500000,

2020-02-16 13:02:39,005 : INFO : PROGRESS: at sentence #2120000, processed 35374651 words, keeping 1154142 word types
2020-02-16 13:02:39,702 : INFO : PROGRESS: at sentence #2130000, processed 35530280 words, keeping 1155222 word types
2020-02-16 13:02:40,197 : INFO : PROGRESS: at sentence #2140000, processed 35640325 words, keeping 1158857 word types
2020-02-16 13:02:40,673 : INFO : PROGRESS: at sentence #2150000, processed 35760330 words, keeping 1161127 word types
2020-02-16 13:02:41,617 : INFO : PROGRESS: at sentence #2160000, processed 35923044 words, keeping 1163211 word types
2020-02-16 13:02:42,403 : INFO : PROGRESS: at sentence #2170000, processed 36066283 words, keeping 1169736 word types
2020-02-16 13:02:42,679 : INFO : PROGRESS: at sentence #2180000, processed 36159893 words, keeping 1171881 word types
2020-02-16 13:02:42,908 : INFO : PROGRESS: at sentence #2190000, processed 36251213 words, keeping 1175786 word types
2020-02-16 13:02:43,333 : INFO : PROGRESS: at sentence #

...

2020-02-16 14:54:24,518 : INFO : EPOCH 20 - PROGRESS: at 18.94% examples, 155420 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:54:25,582 : INFO : EPOCH 20 - PROGRESS: at 19.13% examples, 156078 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:54:26,595 : INFO : EPOCH 20 - PROGRESS: at 19.30% examples, 156558 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:54:27,656 : INFO : EPOCH 20 - PROGRESS: at 19.52% examples, 157492 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:54:28,702 : INFO : EPOCH 20 - PROGRESS: at 19.78% examples, 158835 words/s, in_qsize 0, out_qsize 1
2020-02-16 14:54:29,723 : INFO : EPOCH 20 - PROGRESS: at 20.04% examples, 160258 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:54:30,762 : INFO : EPOCH 20 - PROGRESS: at 20.30% examples, 161630 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:54:31,838 : INFO : EPOCH 20 - PROGRESS: at 20.55% examples, 162784 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:54:32,881 : INFO : EPOCH 20 - PROGRESS: at 20.75% examples, 163522 words/s

2020-02-16 14:55:39,727 : INFO : EPOCH 20 - PROGRESS: at 44.46% examples, 194245 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:55:40,767 : INFO : EPOCH 20 - PROGRESS: at 45.29% examples, 194381 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:55:42,076 : INFO : EPOCH 20 - PROGRESS: at 45.66% examples, 193611 words/s, in_qsize 0, out_qsize 2
2020-02-16 14:55:43,088 : INFO : EPOCH 20 - PROGRESS: at 46.32% examples, 193578 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:55:44,122 : INFO : EPOCH 20 - PROGRESS: at 47.07% examples, 193625 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:55:45,185 : INFO : EPOCH 20 - PROGRESS: at 48.55% examples, 193787 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:55:46,222 : INFO : EPOCH 20 - PROGRESS: at 49.51% examples, 193632 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:55:47,285 : INFO : EPOCH 20 - PROGRESS: at 50.47% examples, 193742 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:55:48,335 : INFO : EPOCH 20 - PROGRESS: at 51.41% examples, 193529 words/s

2020-02-16 14:56:56,281 : INFO : EPOCH 20 - PROGRESS: at 88.89% examples, 196169 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:56:57,293 : INFO : EPOCH 20 - PROGRESS: at 89.02% examples, 196093 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:56:58,333 : INFO : EPOCH 20 - PROGRESS: at 89.16% examples, 196038 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:56:59,339 : INFO : EPOCH 20 - PROGRESS: at 89.33% examples, 196151 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:57:00,351 : INFO : EPOCH 20 - PROGRESS: at 89.48% examples, 196186 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:57:01,372 : INFO : EPOCH 20 - PROGRESS: at 89.51% examples, 195612 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:57:02,823 : INFO : EPOCH 20 - PROGRESS: at 89.53% examples, 194707 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:57:04,274 : INFO : EPOCH 20 - PROGRESS: at 89.57% examples, 193916 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:57:05,284 : INFO : EPOCH 20 - PROGRESS: at 89.59% examples, 193296 words/s

2020-02-16 14:58:16,880 : INFO : EPOCH 20 - PROGRESS: at 98.67% examples, 185060 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:58:17,984 : INFO : EPOCH 20 - PROGRESS: at 98.87% examples, 184918 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:58:19,002 : INFO : EPOCH 20 - PROGRESS: at 99.28% examples, 185067 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:58:20,032 : INFO : EPOCH 20 - PROGRESS: at 99.52% examples, 185008 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:58:21,098 : INFO : EPOCH 20 - PROGRESS: at 99.82% examples, 184951 words/s, in_qsize 0, out_qsize 0
2020-02-16 14:58:21,536 : INFO : worker thread finished; awaiting finish of 99 more threads
2020-02-16 14:58:21,634 : INFO : worker thread finished; awaiting finish of 98 more threads
2020-02-16 14:58:21,635 : INFO : worker thread finished; awaiting finish of 97 more threads
2020-02-16 14:58:21,638 : INFO : worker thread finished; awaiting finish of 96 more threads
2020-02-16 14:58:21,640 : INFO : worker thread finished; await

2020-02-16 14:58:21,979 : INFO : worker thread finished; awaiting finish of 16 more threads
2020-02-16 14:58:21,980 : INFO : worker thread finished; awaiting finish of 15 more threads
2020-02-16 14:58:21,981 : INFO : worker thread finished; awaiting finish of 14 more threads
2020-02-16 14:58:21,983 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-02-16 14:58:21,984 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-02-16 14:58:21,986 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-02-16 14:58:21,987 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-02-16 14:58:21,989 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-02-16 14:58:21,990 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-02-16 14:58:21,992 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-02-16 14:58:21,993 : INFO : worker thread finished; awaiting finish of 6 more 

In [5]:
from gensim.models import Word2Vec,KeyedVectors
am_model= KeyedVectors.load('Model/5w_10ng_am_w2v_cbow_200D')
print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ ጅማ".split()))
print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ".split()))
print(am_model.wv.doesnt_match("ጥረቶች ተግባራት ቋንቋዎች ክንዋኔዎች".split()))
print(am_model.wv.most_similar_cosmul(positive=['ችግር', 'አገልግሎቶች'], negative=['አገልግሎት']))
print(am_model.wv.most_similar_cosmul(positive=['ቱርክ', 'አፍሪቃ'], negative=['ኢትዮጵያ']))
print(am_model.wv.most_similar_cosmul(positive=['እናት', 'አባቶች'], negative=['አባት']))
# 			እናቶች					
# 		የክርስቲያኖችን				
# 						ውጤታማነት
#							ጀርመንኛ
#								
# 			የቶጎ				ሄልሲንኪ		
#		አገልግሎቶች				
print(am_model.wv.most_similar_cosmul(positive=['ጀርመን', 'ኦሮምኛ'], negative=['ኦሮሞ']))
print(am_model.wv.most_similar_cosmul(positive=['የኢትዮጵያ', 'በርሊን'], negative=['የጀርመን']))
print(am_model.wv.most_similar_cosmul(positive=['የፊንላንድ', 'የቶጎ'], negative=['ሎሜ']))
print(am_model.wv.most_similar_cosmul(positive=['አገልግሎት', 'መምህራኖች'], negative=['መምህር']))

print(am_model.wv.most_similar_cosmul(positive=['ውጤት', 'ምርታማነት'], negative=['ምርት']))

2020-02-16 15:04:28,491 : INFO : loading Word2VecKeyedVectors object from Model/5w_10ng_am_w2v_cbow_200D
2020-02-16 15:04:33,548 : INFO : loading wv recursively from Model/5w_10ng_am_w2v_cbow_200D.wv.* with mmap=None
2020-02-16 15:04:33,551 : INFO : loading vectors from Model/5w_10ng_am_w2v_cbow_200D.wv.vectors.npy with mmap=None
2020-02-16 15:04:34,144 : INFO : setting ignored attribute vectors_norm to None
2020-02-16 15:04:34,147 : INFO : loading vocabulary recursively from Model/5w_10ng_am_w2v_cbow_200D.vocabulary.* with mmap=None
2020-02-16 15:04:34,149 : INFO : loading trainables recursively from Model/5w_10ng_am_w2v_cbow_200D.trainables.* with mmap=None
2020-02-16 15:04:34,150 : INFO : loading syn1neg from Model/5w_10ng_am_w2v_cbow_200D.trainables.syn1neg.npy with mmap=None
2020-02-16 15:04:34,681 : INFO : setting ignored attribute cum_table to None
2020-02-16 15:04:34,684 : INFO : loaded Model/5w_10ng_am_w2v_cbow_200D
2020-02-16 15:04:37,954 : INFO : precomputing L2-norms of wor

ጅማ
ብዙ
ቋንቋዎች
[('ችግሮች', 0.9953346848487854), ('ቀውሶች', 0.9248161315917969), ('ችግርም', 0.9183478951454163), ('ችግሮችም', 0.8929307460784912), ('ልዩነቶች', 0.8796794414520264), ('ችግሮችን', 0.8784118294715881), ('ተግዳሮቶች', 0.8752264976501465), ('ቀውስ', 0.8713561296463013), ('ጉድለቶች', 0.8710977435112), ('ጉዳቶች', 0.8673685193061829)]
[('እስያና', 0.8499000668525696), ('አየርላንድ', 0.8438536524772644), ('ፓስፊክና', 0.827453076839447), ('ሱማትራ', 0.8244550824165344), ('ማሌዢያ', 0.8235152363777161), ('ጃቫ', 0.8229060769081116), ('ህንድ', 0.8199477195739746), ('ቬትናም', 0.8182960748672485), ('እስያ', 0.8167277574539185), ('የሜዲቴራኒያን', 0.8147426843643188)]
[('ወጣቶች', 0.780503511428833), ('ልጆቿ', 0.7785176634788513), ('እህቶች', 0.778070330619812), ('ኢትዮጵያውያን', 0.7765077948570251), ('ሴቶች', 0.7693566679954529), ('እናቶች', 0.7630278468132019), ('እህቶችም', 0.7629272937774658), ('ቀሳውስት', 0.7618855237960815), ('ምእመናን', 0.7593709826469421), ('አገልጋዮች', 0.7560131549835205)]
[('ጃፓን', 0.9203752279281616), ('አውሮፓ', 0.9117605686187744), ('ላቲንና', 0.90663

In [3]:
from gensim.models import Word2Vec,KeyedVectors
am_model= KeyedVectors.load('Model/10_sample_am_w2v_cbow_300D')
print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ ጅማ".split()))
print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ".split()))
print(am_model.wv.doesnt_match("ጥረቶች ተግባራት ቋንቋዎች ክንዋኔዎች".split()))
print(am_model.wv.most_similar_cosmul(positive=['ችግር', 'አገልግሎቶች'], negative=['አገልግሎት']))
print(am_model.wv.most_similar_cosmul(positive=['ቱርክ', 'አፍሪቃ'], negative=['ኢትዮጵያ']))
print(am_model.wv.most_similar_cosmul(positive=['እናት', 'አባቶች'], negative=['አባት']))
# 			እናቶች					
# 		የክርስቲያኖችን				
# 						ውጤታማነት
#							ጀርመንኛ
#								
# 			የቶጎ				ሄልሲንኪ		
#		አገልግሎቶች				
print(am_model.wv.most_similar_cosmul(positive=['ጀርመን', 'ኦሮምኛ'], negative=['ኦሮሞ']))
print(am_model.wv.most_similar_cosmul(positive=['የኢትዮጵያ', 'በርሊን'], negative=['የጀርመን']))
print(am_model.wv.most_similar_cosmul(positive=['የፊንላንድ', 'የቶጎ'], negative=['ሎሜ']))
print(am_model.wv.most_similar_cosmul(positive=['አገልግሎት', 'መምህራኖች'], negative=['መምህር']))

print(am_model.wv.most_similar_cosmul(positive=['ውጤት', 'ምርታማነት'], negative=['ምርት']))

2020-02-16 12:57:50,810 : INFO : loading Word2VecKeyedVectors object from Model/10_sample_am_w2v_cbow_300D
2020-02-16 12:57:54,869 : INFO : loading wv recursively from Model/10_sample_am_w2v_cbow_300D.wv.* with mmap=None
2020-02-16 12:57:54,872 : INFO : loading vectors from Model/10_sample_am_w2v_cbow_300D.wv.vectors.npy with mmap=None
2020-02-16 12:58:01,614 : INFO : setting ignored attribute vectors_norm to None
2020-02-16 12:58:01,616 : INFO : loading vocabulary recursively from Model/10_sample_am_w2v_cbow_300D.vocabulary.* with mmap=None
2020-02-16 12:58:01,617 : INFO : loading trainables recursively from Model/10_sample_am_w2v_cbow_300D.trainables.* with mmap=None
2020-02-16 12:58:01,618 : INFO : loading syn1neg from Model/10_sample_am_w2v_cbow_300D.trainables.syn1neg.npy with mmap=None
2020-02-16 12:58:08,358 : INFO : setting ignored attribute cum_table to None
2020-02-16 12:58:08,360 : INFO : loaded Model/10_sample_am_w2v_cbow_300D
2020-02-16 12:58:08,458 : INFO : precomputing L

ብዙ
ብዙ
ቋንቋዎች
[('ችግሮች', 0.9509038925170898), ('ችግርም', 0.9285620450973511), ('ችግሮችም', 0.9012255072593689), ('ቀውሶች', 0.8966010212898254), ('መንስኤዎች', 0.8763713240623474), ('ጉድለቶች', 0.8759945034980774), ('ችግሩ', 0.8759174942970276), ('እንከኖች', 0.8690509796142578), ('ውጥረቶች', 0.8614413738250732), ('ችግርና', 0.856182336807251)]
[('፫፼፸፻', 0.8626731038093567), ('እስያና', 0.8505337238311768), ('በሰሜናዊ', 0.8374099731445312), ('አየርላንድ', 0.8366307020187378), ('ፓኪስታን', 0.8268925547599792), ('ቀርጤስ', 0.826856255531311), ('እስያ', 0.8194982409477234), ('የባልሻሚን', 0.8187791109085083), ('ሆባ', 0.8164778351783752), ('አካላን', 0.815757691860199)]
[('አባቶችና', 0.807296097278595), ('ልጆቿ', 0.7943066954612732), ('በጉያዋ', 0.7935752868652344), ('ቤተክርስቲያኗ', 0.7798409461975098), ('ልጆቿን', 0.7732971906661987), ('ቤተክርስቲያናችን', 0.7657631039619446), ('እናቶች', 0.7638204097747803), ('ቤተክርስቲያኒቱ', 0.7610462307929993), ('ተጠብቃ', 0.7600744366645813), ('መነኮሳይያት', 0.7593830823898315)]
[('ጀርመንና', 0.9237395524978638), ('በስፔን', 0.9140588045120239), (

In [3]:
import logging
import os
from gensim.models import Word2Vec,KeyedVectors
import sentencepiece as spm

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
EMBEDDING_DIR='Model/'
PREPROCESSED_DIR='Data/preprocessed/'
class WordEmbeddingConfig(object):
    """Word2Vec Training parameters"""
    window=5 #Maximum skip length window between words
    emb_dim=100 # Set size of word vectors
    emb_lr=0.05 #learning rate for SGD estimation.
    nepoach=20 #number of training epochs
    nthread=100 #number of training threads
    sample = 0 #Set threshold for occurrence of words. Those that appear with higher frequency in the training data will be randomly down-sampled
    negative = 15 #negative sampling is used with defined negative example
    hs = 0 #0 Use Hierarchical Softmax; default is 0 (not used)
    binary=0 # 0 means not saved as .bin. Change to 1 if allowed to binary format
    sg=0 # 0 means CBOW model is used. Change to 1 to use Skip-gram model
    iterate=10 # Run more training iterations
    minFreq=5 #This will discard words that appear less than minFreq times 
    if sg==0:
        model_name='sp_am_w2v_cbow_'+str(emb_dim)+'D'
    elif sg==1:
         model_name='sp_am_w2v_sg_'+str(emb_dim)+'D'

class corpus_sentences(object):# accept sentence stored one per line in list of files inside defined directory
    def __init__(self, dirname):
        self.dirname = dirname
        self.sp=spm.SentencePieceProcessor()
        self.sp_model=self.sp.Load("amh_sp.model")
    
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding='utf8'):
                yield self.sp.EncodeAsPieces(line)
    
def train_w2v_model():
    print('Loading Sentences with memory freindly iterator ...\n')
    sentences = corpus_sentences(PREPROCESSED_DIR) # a memory-friendly iterator 
    if WordEmbeddingConfig.sg==0:
            model_type='CBOW'
    else:
        model_type='Skip-gram'    
    print('Training Sentence Piece Word2Vec '+model_type+' with '+str(WordEmbeddingConfig.emb_dim)+' dimension\n') 
    _model = Word2Vec(sentences, size=WordEmbeddingConfig.emb_dim, window=WordEmbeddingConfig.window, 
                            min_count=WordEmbeddingConfig.minFreq, workers=WordEmbeddingConfig.nthread,sg=WordEmbeddingConfig.sg,
                            iter=WordEmbeddingConfig.iterate,negative=WordEmbeddingConfig.negative,
                            hs=WordEmbeddingConfig.hs,sorted_vocab=1)
  
    #trim unneeded model memory = use (much) less RAM
    _model.init_sims(replace=True)
    
    #Saving model   
    model_name=EMBEDDING_DIR+WordEmbeddingConfig.model_name
    _model.save(model_name)

    return _model        
train_w2v_model()

2020-02-13 21:42:08,060 : INFO : collecting all words and their counts
2020-02-13 21:42:08,075 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Loading Sentences with memory freindly iterator ...

Training Sentence Piece Word2Vec CBOW with 100 dimension



2020-02-13 21:42:08,761 : INFO : PROGRESS: at sentence #10000, processed 231072 words, keeping 18163 word types
2020-02-13 21:42:09,484 : INFO : PROGRESS: at sentence #20000, processed 482259 words, keeping 22981 word types
2020-02-13 21:42:09,936 : INFO : PROGRESS: at sentence #30000, processed 650805 words, keeping 25556 word types
2020-02-13 21:42:10,438 : INFO : PROGRESS: at sentence #40000, processed 833666 words, keeping 26831 word types
2020-02-13 21:42:10,985 : INFO : PROGRESS: at sentence #50000, processed 1037744 words, keeping 27438 word types
2020-02-13 21:42:11,648 : INFO : PROGRESS: at sentence #60000, processed 1275066 words, keeping 28256 word types
2020-02-13 21:42:12,109 : INFO : PROGRESS: at sentence #70000, processed 1447309 words, keeping 28656 word types
2020-02-13 21:42:12,555 : INFO : PROGRESS: at sentence #80000, processed 1604939 words, keeping 29030 word types
2020-02-13 21:42:12,975 : INFO : PROGRESS: at sentence #90000, processed 1761726 words, keeping 2922

2020-02-13 21:42:43,667 : INFO : PROGRESS: at sentence #730000, processed 12003500 words, keeping 31236 word types
2020-02-13 21:42:44,145 : INFO : PROGRESS: at sentence #740000, processed 12162335 words, keeping 31236 word types
2020-02-13 21:42:44,583 : INFO : PROGRESS: at sentence #750000, processed 12318264 words, keeping 31236 word types
2020-02-13 21:42:45,015 : INFO : PROGRESS: at sentence #760000, processed 12473621 words, keeping 31237 word types
2020-02-13 21:42:45,493 : INFO : PROGRESS: at sentence #770000, processed 12630072 words, keeping 31241 word types
2020-02-13 21:42:45,925 : INFO : PROGRESS: at sentence #780000, processed 12781742 words, keeping 31248 word types
2020-02-13 21:42:46,385 : INFO : PROGRESS: at sentence #790000, processed 12936946 words, keeping 31251 word types
2020-02-13 21:42:46,821 : INFO : PROGRESS: at sentence #800000, processed 13091084 words, keeping 31251 word types
2020-02-13 21:42:47,251 : INFO : PROGRESS: at sentence #810000, processed 132397

2020-02-13 21:43:33,168 : INFO : PROGRESS: at sentence #1440000, processed 30578079 words, keeping 31564 word types
2020-02-13 21:43:33,884 : INFO : PROGRESS: at sentence #1450000, processed 30856833 words, keeping 31564 word types
2020-02-13 21:43:34,626 : INFO : PROGRESS: at sentence #1460000, processed 31135096 words, keeping 31566 word types
2020-02-13 21:43:35,407 : INFO : PROGRESS: at sentence #1470000, processed 31414589 words, keeping 31566 word types
2020-02-13 21:43:36,169 : INFO : PROGRESS: at sentence #1480000, processed 31687624 words, keeping 31566 word types
2020-02-13 21:43:36,897 : INFO : PROGRESS: at sentence #1490000, processed 31957034 words, keeping 31566 word types
2020-02-13 21:43:37,629 : INFO : PROGRESS: at sentence #1500000, processed 32223378 words, keeping 31566 word types
2020-02-13 21:43:38,353 : INFO : PROGRESS: at sentence #1510000, processed 32489925 words, keeping 31566 word types
2020-02-13 21:43:39,072 : INFO : PROGRESS: at sentence #1520000, process

...

2020-02-13 22:53:22,502 : INFO : EPOCH 10 - PROGRESS: at 26.79% examples, 221039 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:53:23,552 : INFO : EPOCH 10 - PROGRESS: at 26.84% examples, 220561 words/s, in_qsize 1, out_qsize 0
2020-02-13 22:53:24,599 : INFO : EPOCH 10 - PROGRESS: at 26.92% examples, 220467 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:53:25,634 : INFO : EPOCH 10 - PROGRESS: at 27.00% examples, 220442 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:53:26,653 : INFO : EPOCH 10 - PROGRESS: at 27.08% examples, 220282 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:53:27,673 : INFO : EPOCH 10 - PROGRESS: at 27.15% examples, 220068 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:53:28,693 : INFO : EPOCH 10 - PROGRESS: at 27.23% examples, 220010 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:53:29,694 : INFO : EPOCH 10 - PROGRESS: at 27.30% examples, 219928 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:53:30,719 : INFO : EPOCH 10 - PROGRESS: at 27.37% examples, 219765 words/s

2020-02-13 22:54:37,020 : INFO : EPOCH 10 - PROGRESS: at 55.18% examples, 213375 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:54:38,046 : INFO : EPOCH 10 - PROGRESS: at 56.19% examples, 213297 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:54:39,049 : INFO : EPOCH 10 - PROGRESS: at 56.83% examples, 213262 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:54:40,074 : INFO : EPOCH 10 - PROGRESS: at 57.53% examples, 213212 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:54:41,123 : INFO : EPOCH 10 - PROGRESS: at 58.16% examples, 213144 words/s, in_qsize 1, out_qsize 0
2020-02-13 22:54:42,148 : INFO : EPOCH 10 - PROGRESS: at 58.82% examples, 213169 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:54:43,308 : INFO : EPOCH 10 - PROGRESS: at 59.48% examples, 213082 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:54:44,319 : INFO : EPOCH 10 - PROGRESS: at 60.09% examples, 213011 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:54:45,360 : INFO : EPOCH 10 - PROGRESS: at 60.69% examples, 212879 words/s

2020-02-13 22:55:51,378 : INFO : EPOCH 10 - PROGRESS: at 87.94% examples, 213317 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:55:52,382 : INFO : EPOCH 10 - PROGRESS: at 88.07% examples, 213397 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:55:53,395 : INFO : EPOCH 10 - PROGRESS: at 88.19% examples, 213417 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:55:54,430 : INFO : EPOCH 10 - PROGRESS: at 88.33% examples, 213477 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:55:55,456 : INFO : EPOCH 10 - PROGRESS: at 88.46% examples, 213571 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:55:56,479 : INFO : EPOCH 10 - PROGRESS: at 88.59% examples, 213638 words/s, in_qsize 0, out_qsize 1
2020-02-13 22:55:57,521 : INFO : EPOCH 10 - PROGRESS: at 88.73% examples, 213694 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:55:58,546 : INFO : EPOCH 10 - PROGRESS: at 88.86% examples, 213759 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:55:59,588 : INFO : EPOCH 10 - PROGRESS: at 89.00% examples, 213840 words/s

2020-02-13 22:57:05,063 : INFO : EPOCH 10 - PROGRESS: at 98.23% examples, 216457 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:57:06,072 : INFO : EPOCH 10 - PROGRESS: at 98.37% examples, 216506 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:57:07,095 : INFO : EPOCH 10 - PROGRESS: at 98.53% examples, 216614 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:57:08,110 : INFO : EPOCH 10 - PROGRESS: at 98.69% examples, 216683 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:57:09,135 : INFO : EPOCH 10 - PROGRESS: at 98.92% examples, 216681 words/s, in_qsize 1, out_qsize 0
2020-02-13 22:57:10,164 : INFO : EPOCH 10 - PROGRESS: at 99.18% examples, 216678 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:57:11,175 : INFO : EPOCH 10 - PROGRESS: at 99.41% examples, 216687 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:57:12,212 : INFO : EPOCH 10 - PROGRESS: at 99.66% examples, 216726 words/s, in_qsize 0, out_qsize 0
2020-02-13 22:57:13,399 : INFO : EPOCH 10 - PROGRESS: at 99.95% examples, 216686 words/s

2020-02-13 22:57:13,732 : INFO : worker thread finished; awaiting finish of 21 more threads
2020-02-13 22:57:13,732 : INFO : worker thread finished; awaiting finish of 20 more threads
2020-02-13 22:57:13,733 : INFO : worker thread finished; awaiting finish of 19 more threads
2020-02-13 22:57:13,734 : INFO : worker thread finished; awaiting finish of 18 more threads
2020-02-13 22:57:13,734 : INFO : worker thread finished; awaiting finish of 17 more threads
2020-02-13 22:57:13,735 : INFO : worker thread finished; awaiting finish of 16 more threads
2020-02-13 22:57:13,736 : INFO : worker thread finished; awaiting finish of 15 more threads
2020-02-13 22:57:13,737 : INFO : worker thread finished; awaiting finish of 14 more threads
2020-02-13 22:57:13,737 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-02-13 22:57:13,738 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-02-13 22:57:13,739 : INFO : worker thread finished; awaiting finish of 11 m

In [1]:
from gensim.models import Word2Vec,KeyedVectors
am_model= KeyedVectors.load('Model/5freq_am_w2v_cbow_200D')
# print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ ጅማ".split()))
# print(am_model.wv.doesnt_match("አንድ ሺህ ሚሊዮን አምስት ብዙ".split()))
# print(am_model.wv.doesnt_match("ጥረቶች ተግባራት ቋንቋዎች ክንዋኔዎች".split()))
# print(am_model.wv.most_similar_cosmul(positive=['ችግር', 'አገልግሎቶች'], negative=['አገልግሎት']))
# print(am_model.wv.most_similar_cosmul(positive=['ቱርክ', 'አፍሪቃ'], negative=['ኢትዮጵያ']))
# print(am_model.wv.most_similar_cosmul(positive=['እናት', 'አባቶች'], negative=['አባት']))
# # 			እናቶች					
# # 		የክርስቲያኖችን				
# # 						ውጤታማነት
# #							ጀርመንኛ
# #								
# # 			የቶጎ				ሄልሲንኪ		
# #		አገልግሎቶች				
# print(am_model.wv.most_similar_cosmul(positive=['ጀርመን', 'ኦሮምኛ'], negative=['ኦሮሞ']))
# print(am_model.wv.most_similar_cosmul(positive=['የኢትዮጵያ', 'በርሊን'], negative=['የጀርመን']))
# print(am_model.wv.most_similar_cosmul(positive=['የፊንላንድ', 'የቶጎ'], negative=['ሎሜ']))
# print(am_model.wv.most_similar_cosmul(positive=['አገልግሎት', 'መምህራኖች'], negative=['መምህር']))

# print(am_model.wv.most_similar_cosmul(positive=['ውጤት', 'ምርታማነት'], negative=['ምርት']))
print('Total Vocab: ',len(am_model.wv.vocab))
print(am_model.wv.vocab)

C:\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# !pip install glove_python
import logging
import os
import fasttext

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
EMBEDDING_DIR='Model/'
PREPROCESSED_DIR='Data/preprocessed/'

class corpus_sentences(object):# accept sentence stored one per line in list of files inside defined directory
    def __init__(self, dirname):
        self.dirname = dirname
    
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding='utf8'):
                yield line
    
# creating a corpus object

model = fasttext.train_unsupervised('Data/preprocessed/', minn=2, maxn=10, dim=300,epoch=10,thread=100, lr=0.5)

In [2]:
from bpemb import BPEmb
bpemb_en = BPEmb(lang="am", dim=50)

downloading https://nlp.h-its.org/bpemb/am/am.wiki.bpe.vs10000.model


100%|█████████████████████████████████████████████████████████████████████████| 425894/425894 [00:43<00:00, 9819.16B/s]


downloading https://nlp.h-its.org/bpemb/am/am.wiki.bpe.vs10000.d50.w2v.bin.tar.gz


100%|██████████████████████████████████████████████████████████████████████| 1929157/1929157 [01:40<00:00, 12452.66B/s]
C:\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
bpemb_am = BPEmb(lang="am", vs=100000)

BPEmb fallback: am from vocab size 100000 to 50000
downloading https://nlp.h-its.org/bpemb/am/am.wiki.bpe.vs50000.model


100%|██████████████████████████████████████████████████████████████████████| 1301938/1301938 [01:46<00:00, 12204.87B/s]


downloading https://nlp.h-its.org/bpemb/am/am.wiki.bpe.vs50000.d100.w2v.bin.tar.gz


100%|████████████████████████████████████████████████████████████████████| 18930308/18930308 [15:42<00:00, 20077.90B/s]


In [4]:
type(bpemb_am.emb)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [7]:
analogy_scores = bpemb_am.embed('አገልግሎቶች')#
analogy_scores

array([[-0.204496, -0.15438 ,  0.003482, -0.537537,  0.602293, -0.181454,
         0.064071,  0.075879,  0.196366,  0.261775,  0.115011,  0.339124,
        -0.107818,  0.111809,  0.020449,  0.442207,  0.271473,  0.024214,
        -0.336247, -0.010433,  0.010571,  0.320751,  0.122873, -0.05821 ,
        -0.070864,  0.113487, -0.06856 ,  0.046819, -0.074432,  0.165071,
         0.078311, -0.189665, -0.198913,  0.035079,  0.094295,  0.31182 ,
         0.084007, -0.067356,  0.231944,  0.072742,  0.408201, -0.292842,
         0.024934,  0.218651,  0.232899,  0.086072,  0.22353 , -0.163427,
        -0.22133 , -0.167458,  0.331141,  0.093485, -0.023519,  0.045251,
        -0.165391,  0.159142, -0.179416,  0.302601,  0.150757,  0.082018,
         0.097446,  0.193447,  0.034804,  0.494798,  0.145383,  0.024734,
         0.150248,  0.255943,  0.312128,  0.229794, -0.006274,  0.263561,
         0.110268, -0.22932 ,  0.068955, -0.168877,  0.134745,  0.166696,
        -0.17457 ,  0.158557, -0.09499